# HW5

In [2]:
import pymc as pm
import numpy as np
from scipy import stats

In [6]:
# Define the dimensionality of the multivariate normal distribution
p = 10

# Define the scale matrix for the Wishart distribution
Psi = np.eye(p)

# Generate an inverse-Wishart sample for the covariance matrix
a_cov = stats.invwishart(df=p+2, scale=Psi).rvs(1)

# Define the number of data points
n = 1000

# Generate simulated data points from a multivariate normal distribution
y = stats.multivariate_normal(mean=np.zeros(p), cov=a_cov).rvs(size=n)

In [7]:
# Define the PyMC3 model using the LKJCholeskyCov distribution
with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=p), compute_corr=False)
    # Define the Cholesky factor of the covariance matrix

    L = pm.expand_packed_triangular(p, packed_L)

    Sigma = pm.Deterministic('Sigma', L.dot(L.T))  # Reconstructing the covariance matrix

    # Define the prior for the mean vector using a small constant for the covariance for numerical stability
    mu = pm.MvNormal('mu', mu=np.zeros(p), cov=np.eye(p)*1e-6, shape=p)

    # Define the likelihood of the observed data
    y_obs = pm.MvNormal('y_obs', mu=mu, chol=L, observed=y)

# Sampling from the model
with MNV_LKJ:
    trace = pm.sample()

# The trace object now contains the samples for the posterior distribution.